In [1]:
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
import tkinter as tk
from pysrc.simplertplot.plotwidget import RTPlotWidget
from hello.hello3 import HelloApp, TrueError
from time import time

In [3]:
ip = '192.168.1.84'
#ip = '71.189.82.196:86'

In [4]:
root = tk.Tk()
root.withdraw()
p = RTPlotWidget(root, max_pts=3600)
p.tkcanvas.config(height=400, width=600)

In [5]:
h = HelloApp(ip)
h.login()
h.getMainValues()
pass

In [6]:
def actual_rate(n2, o2, co2):
    _, pc_1, _ = pc(0, o2, co2)
    _, pc_2, _ = pc(n2, o2, co2)
    ratio = pc_1/pc_2
    actual_n2 = n2 * ratio
    return actual_n2

def pc(a, b, c):
    t = a + b + c
    return a/t, b/t, c/t

In [7]:
def timer():
    start = time()
    while True:
        current = time()
        yield current - start
get_time = timer().__next__

In [8]:
g_stop_tasks = False
g_after = None
def tk_task(tk, interval, f):
    global g_after, g_stop_tasks
    def fn(*args, **kw):
        try:
            return f(*args, **kw)
        finally:
            if not g_stop_tasks:
                g_after = tk.after(interval, fn)
            else:
                tk.after_cancel(g_after)
    g_after = tk.after(interval, fn)

In [9]:
def update():
    mv = h.getMainValues()
    do = mv['do']
    pv = do['pv']
    t = get_time()
    p.add_xy(t, pv)
    update_labels(mv)

In [10]:
def printdir(o):
    for a in dir(o):
        if not a.startswith("_"):
            print(a, getattr(o, a))

In [11]:
def conf2(ev):
    if ev.widget == root:
        p.tkcanvas.config(height=ev.height, width=ev.width-rbtn.winfo_width())

In [12]:
class ILabel(tk.Label):
    def __init__(self, master, name, value=0, fmt="%.2f"):
        super().__init__(master)
        self.fmt = fmt
        self.name = name
        self.set_value(value)
        
    def set_value(self, val):
        val = self.fmt % val
        self.config(text="%s: %s" % (self.name, val))
   
dos = {}
def mk_label(name, val, row):
    l = ILabel(root, name, val)
    dos[name] = l
    l.grid(row=row, column=1)
    
mk_label("PV", 0, 1)
mk_label("O2 Flow %", 0, 2)
mk_label("O2 Flow LPM", 0, 3)
    
def update_labels(mv):
    while True:
        try:
            av = h.getAdvancedValues()
        except TrueError:
            pass
        else:
            break

    dos['PV'].set_value(mv['do']['pv'])
    dos['O2 Flow %'].set_value(av['O2 Flow (%)'])
    dos['O2 Flow LPM'].set_value(av['MFCO2FlowFeedback(LPM)'])
    

In [ ]:
p.grid(rowspan=20, column=0, row=0)    
rbtn = tk.Button(root, text="Reset", command=lambda: p.reset())
rbtn.grid(column=1, row=0, sticky="ew")
root.bind("<Configure>", conf2)
p.set_xaxis_text("Time (s)")
p.set_yaxis_text("DO PV (%)")
def on_closing():
    from tkinter import messagebox
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        global g_stop_tasks
        g_stop_tasks = True
        root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.wm_deiconify()
tk_task(root, 500, update)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\program files\python35\lib\tkinter\__init__.py", line 1550, in __call__
    return self.func(*args)
  File "c:\program files\python35\lib\tkinter\__init__.py", line 596, in callit
    func(*args)
  File "<ipython-input-8-a36d465b896c>", line 7, in fn
    return f(*args, **kw)
  File "<ipython-input-9-390824dfceab>", line 7, in update
    update_labels(mv)
  File "<ipython-input-12-fb64b880de87>", line 25, in update_labels
    av = h.getAdvancedValues()
AttributeError: 'HelloApp' object has no attribute 'getAdvancedValues'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\program files\python35\lib\tkinter\__init__.py", line 1550, in __call__
    return self.func(*args)
  File "c:\program files\python35\lib\tkinter\__init__.py", line 596, in callit
    func(*args)
  File "<ipython-input-8-a36d465b896c>", line 7, in fn
    return f(*args, **kw)
  File "<ipython-input-9-390824dfceab>", li